In [2]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {
    "collapsed": false
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "data7804\r\n"
     ]
    }
   ],
   "source": [
    "# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原\n",
    "# View dataset directory. \n",
    "# This directory will be recovered automatically after resetting environment. \n",
    "!ls /home/aistudio/data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.\n",
    "# View personal work directory. \n",
    "# All changes under this directory will be kept even after reset. \n",
    "# Please clean unnecessary files in time to speed up environment loading. \n",
    "!ls /home/aistudio/work"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: \n",
    "# Also add the following code, \n",
    "# so that every time the environment (kernel) starts, \n",
    "# just run the following code: \n",
    "import sys \n",
    "sys.path.append('/home/aistudio/external-libraries')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "#加载飞桨、Numpy和相关类库\n",
    "import paddle\n",
    "from paddle.nn import Linear\n",
    "import paddle.nn.functional as F\n",
    "import numpy as np\n",
    "import os\n",
    "import random"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "collapsed": false
   },
   "source": [
    "请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>\n",
    "Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "def load_data():\r\n",
    "    # 从文件导入数据\r\n",
    "    datafile = '/home/aistudio/data/data7804/housing.data'\r\n",
    "    data = np.fromfile(datafile, sep=' ', dtype=np.float32)\r\n",
    "\r\n",
    "    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数\r\n",
    "    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \\\r\n",
    "                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]\r\n",
    "    feature_num = len(feature_names)\r\n",
    "\r\n",
    "    # 将原始数据进行Reshape，变成[N, 14]这样的形状\r\n",
    "    data = data.reshape([data.shape[0] // feature_num, feature_num])\r\n",
    "\r\n",
    "    # 将原数据集拆分成训练集和测试集\r\n",
    "    # 这里使用80%的数据做训练，20%的数据做测试\r\n",
    "    # 测试集和训练集必须是没有交集的\r\n",
    "    ratio = 0.8\r\n",
    "    offset = int(data.shape[0] * ratio)\r\n",
    "    training_data = data[:offset]\r\n",
    "\r\n",
    "    # 计算train数据集的最大值，最小值，平均值\r\n",
    "    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \\\r\n",
    "                                 training_data.sum(axis=0) / training_data.shape[0]\r\n",
    "    \r\n",
    "    # 记录数据的归一化参数，在预测时对数据做归一化\r\n",
    "    global max_values\r\n",
    "    global min_values\r\n",
    "    global avg_values\r\n",
    "    max_values = maximums\r\n",
    "    min_values = minimums\r\n",
    "    avg_values = avgs\r\n",
    "\r\n",
    "    # 对数据进行归一化处理\r\n",
    "    for i in range(feature_num):\r\n",
    "        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])\r\n",
    "\r\n",
    "    # 训练集和测试集的划分比例\r\n",
    "    training_data = data[:offset]\r\n",
    "    test_data = data[offset:]\r\n",
    "    return training_data, test_data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "class Regressor(paddle.nn.Layer):\r\n",
    "\r\n",
    "    # self代表类的实例自身\r\n",
    "    def __init__(self):\r\n",
    "        # 初始化父类中的一些参数\r\n",
    "        super(Regressor, self).__init__()\r\n",
    "        \r\n",
    "        # 定义一层全连接层，输入维度是13，输出维度是1\r\n",
    "        self.fc = Linear(in_features=13, out_features=1)\r\n",
    "    \r\n",
    "    # 网络的前向计算\r\n",
    "    def forward(self, inputs):\r\n",
    "        x = self.fc(inputs)\r\n",
    "        return x"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "# 声明定义好的线性回归模型\r\n",
    "model = Regressor()\r\n",
    "# 开启模型训练模式\r\n",
    "model.train()\r\n",
    "# 加载数据\r\n",
    "training_data, test_data = load_data()\r\n",
    "# 定义优化算法，使用随机梯度下降SGD\r\n",
    "# 学习率设置为0.01\r\n",
    "opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {
    "collapsed": false
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "epoch: 0, iter: 0, loss is: [0.07326401]\n",
      "epoch: 0, iter: 20, loss is: [0.06736588]\n",
      "epoch: 0, iter: 40, loss is: [0.07595865]\n",
      "epoch: 1, iter: 0, loss is: [0.06000581]\n",
      "epoch: 1, iter: 20, loss is: [0.10520406]\n",
      "epoch: 1, iter: 40, loss is: [0.08941956]\n",
      "epoch: 2, iter: 0, loss is: [0.03488638]\n",
      "epoch: 2, iter: 20, loss is: [0.10645159]\n",
      "epoch: 2, iter: 40, loss is: [0.0421632]\n",
      "epoch: 3, iter: 0, loss is: [0.02854164]\n",
      "epoch: 3, iter: 20, loss is: [0.09655952]\n",
      "epoch: 3, iter: 40, loss is: [0.06166927]\n",
      "epoch: 4, iter: 0, loss is: [0.02804949]\n",
      "epoch: 4, iter: 20, loss is: [0.01197303]\n",
      "epoch: 4, iter: 40, loss is: [0.03577208]\n",
      "epoch: 5, iter: 0, loss is: [0.03179492]\n",
      "epoch: 5, iter: 20, loss is: [0.01415187]\n",
      "epoch: 5, iter: 40, loss is: [0.03297611]\n",
      "epoch: 6, iter: 0, loss is: [0.05811074]\n",
      "epoch: 6, iter: 20, loss is: [0.07061922]\n",
      "epoch: 6, iter: 40, loss is: [0.01294551]\n",
      "epoch: 7, iter: 0, loss is: [0.04048797]\n",
      "epoch: 7, iter: 20, loss is: [0.06761952]\n",
      "epoch: 7, iter: 40, loss is: [0.03477216]\n",
      "epoch: 8, iter: 0, loss is: [0.01989347]\n",
      "epoch: 8, iter: 20, loss is: [0.05404834]\n",
      "epoch: 8, iter: 40, loss is: [0.20299487]\n",
      "epoch: 9, iter: 0, loss is: [0.0555575]\n",
      "epoch: 9, iter: 20, loss is: [0.02317487]\n",
      "epoch: 9, iter: 40, loss is: [0.02131323]\n"
     ]
    }
   ],
   "source": [
    "EPOCH_NUM = 10   # 设置外层循环次数\r\n",
    "BATCH_SIZE = 10  # 设置batch大小\r\n",
    "\r\n",
    "# 定义外层循环\r\n",
    "for epoch_id in range(EPOCH_NUM):\r\n",
    "    # 在每轮迭代开始之前，将训练数据的顺序随机的打乱\r\n",
    "    np.random.shuffle(training_data)\r\n",
    "    # 将训练数据进行拆分，每个batch包含10条数据\r\n",
    "    mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]\r\n",
    "    # 定义内层循环\r\n",
    "    for iter_id, mini_batch in enumerate(mini_batches):\r\n",
    "        x = np.array(mini_batch[:, :-1]) # 获得当前批次训练数据\r\n",
    "        y = np.array(mini_batch[:, -1:]) # 获得当前批次训练标签（真实房价）\r\n",
    "        # 将numpy数据转为飞桨动态图tensor形式\r\n",
    "        house_features = paddle.to_tensor(x)\r\n",
    "        prices = paddle.to_tensor(y)\r\n",
    "        \r\n",
    "        # 前向计算\r\n",
    "        predicts = model(house_features)\r\n",
    "        \r\n",
    "        # 计算损失\r\n",
    "        loss = F.square_error_cost(predicts, label=prices)\r\n",
    "        avg_loss = paddle.mean(loss)\r\n",
    "        if iter_id%20==0:\r\n",
    "            print(\"epoch: {}, iter: {}, loss is: {}\".format(epoch_id, iter_id, avg_loss.numpy()))\r\n",
    "        \r\n",
    "        # 反向传播\r\n",
    "        avg_loss.backward()\r\n",
    "        # 最小化loss,更新参数\r\n",
    "        opt.step()\r\n",
    "        # 清除梯度\r\n",
    "        opt.clear_grad()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {
    "collapsed": false
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "模型保存成功，模型参数保存在LR_model.pdparams中\n"
     ]
    }
   ],
   "source": [
    "# 保存模型参数，文件名为LR_model.pdparams\r\n",
    "paddle.save(model.state_dict(), 'LR_model.pdparams')\r\n",
    "print(\"模型保存成功，模型参数保存在LR_model.pdparams中\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "def load_one_example():\r\n",
    "    # 从上边已加载的测试集中，随机选择一条作为测试数据\r\n",
    "    idx = np.random.randint(0, test_data.shape[0])\r\n",
    "    idx = -10\r\n",
    "    one_data, label = test_data[idx, :-1], test_data[idx, -1]\r\n",
    "    # 修改该条数据shape为[1,13]\r\n",
    "    one_data =  one_data.reshape([1,-1])\r\n",
    "\r\n",
    "    return one_data, label"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "PaddlePaddle 2.1.0 (Python 3.5)",
   "language": "python",
   "name": "py35-paddle1.2.0"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 1
}

NameError: name 'false' is not defined

In [0]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 